In [ ]:
# initial setup
%run "../../../common/0_notebooks_base_setup.py"


---

<img src='../../../common/logo_DH.png' align='left' width=35%/>


# Preprocesamiento del dataset

#### Comentario:

En esta notebook prepararemos el dataset para el desafío de la clase. Al final de esta notebook se genera el archivo 'dataset_preprocessed.csv' que ya se encuentra en la carpeta 'Datos'. 

Esta notebook sirve para repasar limpieza, imputación, manejo de fechas y joins con pandas. Se sugiere hacer la práctica de la clase partiendo del dataset ya preprocesado para ver los temas de feature selection y balance de clases y que esta notebook la revisen fuera del horario de clase.

---

Trabajaremos con un dataset de campañas de marketing. El mismo está representado por un conjunto de tablas relacionales:

* train: relaciona id de consumidores, cupones de descuento, campaña de marketing y contiene la variable target 'redemption_status', que dice si el cupón fue canjeado o no.

* campaign data: fecha de inicio y finalización de la campaña y tipo de campaña

* customer_demographics: rango etareo, estado civil, tamaño de la familia, nro de hijos, nivel de ingresos, propietario/inquilino/hipoteca

* coupon_item_mapping: cada cupón da beneficios sobre distintos items

* item_data: marca, tipo de marca y categoría del item

* customer_transaction_data: datos sobre las compras hechas por los consumidores. No sabemos en qué compra usaron el cupón de descuento, tal vez la promoción sirve para más de una compra.

<img src='../Data/marketing/Schema.png'>

<b>Origen del dataset:</b>

https://www.kaggle.com/vasudeva009/coupon-redemption-smote-feature-selection/data

In [ ]:
import pandas as pd
from sklearn.impute import SimpleImputer

In [ ]:
train=pd.read_csv('../Data/marketing/train.csv')
cust_dem=pd.read_csv('../Data/marketing/customer_demographics.csv')
trans=pd.read_csv('../Data/marketing/customer_transaction_data.csv')
coupon_item=pd.read_csv('../Data/marketing/coupon_item_mapping.csv')
item_data=pd.read_csv('../Data/marketing/item_data.csv')
campaign_data=pd.read_csv('../Data/marketing/campaign_data.csv')

---
#### Train dataframe

* Mirar el dataframe y chequear si hay valores nulos

In [ ]:
display(train.head(3))
print('\nHay nulos:',train.isnull().any().any())

---
#### Customer demographics dataframe

* Chequear si hay nulos
* Mirar los dtypes
* Imputar los valores nulos por la moda de las columnas
* Llevar las columnas de variables categóricas a dtype 'category'

In [ ]:
# cust_dem
display(cust_dem.head(3))
print('\n-------------------------')
print('Dtypes:\n') 
print(cust_dem.dtypes)
print('\n-------------------------')
print('Nulos:\n')      
print(cust_dem.isnull().sum())

Imputamos valores faltantes por la moda

In [ ]:
# Imputamos valores faltantes
imp=SimpleImputer(strategy='most_frequent')
cust_dem[['marital_status','no_of_children']]=imp.fit_transform(cust_dem[['marital_status','no_of_children']])
print('Nulos:\n')      
print(cust_dem.isnull().sum())

Llevamos variables a tipo categóricas

In [ ]:
# categorical variables
cust_dem.iloc[:,1:]=cust_dem.iloc[:,1:].astype('category')
print('Dtypes:\n') 
print(cust_dem.dtypes)

---
#### Transactions dataframe
* Chequear si hay nulos
* ver los dtypes
* dropear la columna 'dates'

In [ ]:
# trans data
display(trans.head(3))
print('\nHay nulos:',trans.isnull().any().any())
print('\nDtypes:\n') 
print(trans.dtypes)

In [ ]:
trans=trans.drop('date',axis=1)

In [ ]:
trans.head(3)
print('\nDtypes:\n') 
print(trans.dtypes)

---
#### Coupon-Item dataframe
* chequear si hay nulos

In [ ]:
# Coupon item map
display(coupon_item.head(3))
print('\nHay nulos:',coupon_item.isnull().any().any())

---
#### Item dataframe
* Chequear si hay nulos
* Llevar las columnas categóricas a dtype 'category'

In [ ]:
# Item Data
display(item_data.head(3))
item_data[['brand','brand_type','category']]=item_data[['brand','brand_type','category']].astype('category')
print(item_data.dtypes)
print('\nHay nulos:',item_data.isnull().any().any())

---
#### Campaign dataframe
* Chequear si hay valores nulos
* Llevar las columnas start_date y end_date a tipo TimeStamp
* Calcular el período de las promos como end_date - start_date
* Dropear start_date y end_date

In [ ]:
display(campaign_data.head(3))
print(campaign_data.dtypes)
print('\nHay nulos:',campaign_data.isnull().any().any())

Calculamos cuanto tiempo duró la promoción

In [ ]:
# Calculamos el período de la promoción

start_date=pd.to_datetime(campaign_data['start_date'],dayfirst=True)
end_date=pd.to_datetime(campaign_data['end_date'],dayfirst=True)

campaign_data['duration']=end_date-start_date # Periodo de validez del cupon
campaign_data['duration']=campaign_data['duration'].apply(lambda x: x.days)

campaign_data['campaign_type']=campaign_data['campaign_type'].astype('category')

campaign_data['month']=start_date.apply(lambda x : x.month).astype('category')
campaign_data['year']=start_date.apply(lambda x : x.year).astype('category')


In [ ]:
campaign_data=campaign_data.drop(['start_date','end_date'],axis=1)

display(campaign_data.head(3))
print(campaign_data.dtypes)

---
#### Merge Dataframes
1. Hacer un merge entre train y campaign_data. ¿Qué columna va en 'on'? ¿Qué criterio va en 'how'?
2. Hacer un merge de 1. con customer_demographics. ¿Qué columna va en 'on'? ¿Qué criterio va en 'how'?
3. Chequear valores nulos en 2.
4. Imputamos valores faltantes en las columnas demográficas. Para eso, sampleamos con reposición de la distribución conjunta de atributos demográficos observados.
5. Merge con las transacciones promedio por sujeto:
* Hacer un groupby por consumidor en el dataframe de transaccions
* Agregar el valor medio a las variables continuas ('selling_price','quantity', etc)
* merge con el dataframe obtenido en 4. ¿Qué columna va en 'on'? ¿Qué criterio va en 'how'?
6. Merge con transacciones promedio por item
* a. Hacer un groupby por item en el dataframe de transaccions
* b. Calcular el valor medio de las variables continuas
* c. Hacer un merge con el dataframe 'coupon_item'
* d. Hacer un merge de c. con item_data ¿Qué columna va en 'on'? ¿Qué criterio va en 'how'?
* e. Hacer un groupby del dataframe d. agrupando por 'coupon_id'. Para cada grupo calcular la moda de las variables categóricas 'brand','brand_type,'category'
* f. Hacer un groupby del dataframe d. agrupando por 'coupon_id'. Para cada grupo calcular la media de las variables continuas.
* g. Hacer un merge de d. con e. y luego con f. ¿Qué columna va en 'on'? ¿Qué criterio va en 'how'?
7. Guardar el dataframe resultante en un csv para la práctica de la clase.

In [ ]:
train.shape,campaign_data.shape, cust_dem.shape,trans.shape,coupon_item.shape,item_data.shape

<b>Hacemos un merge entre train y campaign data

In [ ]:
data=pd.merge(train,campaign_data,on='campaign_id',how='left')
print(data.shape)
data.isnull().sum()


<b> Merge con los datos demográficos de los consumidores

In [ ]:
data=pd.merge(data,cust_dem,on='customer_id',how='left')
data.shape

In [ ]:
data.isnull().sum()

<b> Imputamos valores faltentes</b>

Vamos a samplear de la distribución conjunta de variables con valores faltantes. De esta manera conservamos la variabilidad de los datos (no imputamos todo por la moda) y la estructura de correlaciones entre variables (no asignamos un valor de age_range incompatible con uno el de family_size)

In [ ]:
# Imputamos valores faltantes sampleando con reposicion de los datos observados
# De esta manera preservamos la estructura de correlaciones observada entre las columnas demográficas

impute_columns=['age_range','marital_status','rented','family_size','no_of_children','income_bracket']

null_mask=data[impute_columns].isnull().all(axis=1)
random_samples=data.loc[~null_mask,impute_columns].sample(n=null_mask.sum(),replace=True)
data.loc[null_mask,impute_columns]=random_samples.values

data.isnull().sum()

<b> Merge con información de transacciones promedio por cliente</b>

Agregamos la información de la actividad de compras típicas hechas por cada consumidor.

In [ ]:
# Merge transactions: calculo la media de los valores de las transacciones por sujeto
trans_customer=trans.groupby('customer_id')[['quantity','selling_price','other_discount','coupon_discount']].agg('mean')
trans_customer.columns=['mean_'+col+'_cust' for col in trans_customer.columns]
trans_customer=trans_customer.reset_index();

In [ ]:
data=pd.merge(data,trans_customer,on='customer_id',how='left')

In [ ]:
data.shape

In [ ]:
data.head(3)

<b> Transacciones promedio por item </b>

Tenemos el mapeo entre cupón e item. Vamos a agregar información sobre cuánto se consume cada item.

In [ ]:
# Merge with item data
trans_item=trans.groupby('item_id')[['quantity','selling_price','other_discount','coupon_discount']].mean()
trans_item.columns=['mean_'+col+'_item' for col in trans_item.columns]
trans_item=trans_item.reset_index()
trans_item.head(3)


In [ ]:
coupon_item_trans=pd.merge(coupon_item,trans_item,on='item_id',how='inner')
coupon_item_trans_brand=pd.merge(coupon_item_trans,item_data,on='item_id',how='left')
coupon_item_trans_brand.head(3)

In [ ]:
coupon_item_trans_brand.dtypes

In [ ]:
gb=coupon_item_trans_brand.groupby('coupon_id')
coupon_category=gb['category'].apply(lambda x:x.value_counts().index[0])
coupon_brand_type=gb['brand_type'].apply(lambda x:x.value_counts().index[0])
coupon_brand=gb['brand'].apply(lambda x:x.value_counts().index[0])

coupon_brand=pd.concat([pd.DataFrame(coupon_brand),pd.DataFrame(coupon_brand_type),pd.DataFrame(coupon_category)], axis=1).reset_index()
coupon_brand.head(3)

In [ ]:
coupon_item_sales=coupon_item_trans_brand.groupby('coupon_id')[['mean_quantity_item','mean_selling_price_item','mean_other_discount_item','mean_coupon_discount_item']].mean()
coupon_item_sales.columns=[col+'_coupon' for col in coupon_item_sales]
coupon_item_sales=coupon_item_sales.reset_index()
coupon_item_sales.head(3)


In [ ]:
data=pd.merge(data,coupon_item_sales,on='coupon_id',how='left')

In [ ]:
data=pd.merge(data,coupon_brand,on='coupon_id',how='left')

In [ ]:
data[['brand','brand_type','category']]=data[['brand','brand_type','category']].astype('category')
data.dtypes

In [ ]:
data.shape

In [ ]:
# Guardamos los datos
data.to_csv('../Data/marketing/data_preprocessed.csv')